In [4]:
"""
PDB to CRD Converter Unit Tests

This notebook tests the functionality of the PDB to CRD converter package.
"""
import os
import sys
import pytest

# Get the directory where this notebook is located
NOTEBOOK_DIR = os.path.dirname(os.path.abspath("__file__" if "__file__" in locals() else os.getcwd()))
PACKAGE_ROOT = os.path.abspath(os.path.join(NOTEBOOK_DIR, ".."))
TEST_DATA_DIR = os.path.join(PACKAGE_ROOT, "tests", "sample_data")

# Add package root to Python path
sys.path.insert(0, PACKAGE_ROOT)

try:
    from pdb2crd.converter import convert_pdb_to_crd, save_crd
except ImportError as e:
    raise ImportError(f"Could not import converter: {str(e)}\n"
                     "Make sure to either:\n"
                     "1. Install package with 'pip install -e .'\n"
                     "2. Run notebook from correct directory")

# Paths to test files
SAMPLE_PDB = os.path.join(TEST_DATA_DIR, "sample.pdb")
EXPECTED_CRD = os.path.join(TEST_DATA_DIR, "expected_output.crd")
TEST_OUTPUT = os.path.join(TEST_DATA_DIR, "test_output.crd")

print(f"Package root: {PACKAGE_ROOT}")
print(f"Test data directory: {TEST_DATA_DIR}")
print(f"Sample PDB path: {SAMPLE_PDB}")
print(f"Expected CRD path: {EXPECTED_CRD}")

Package root: c:\Users\hlern\code\git
Test data directory: c:\Users\hlern\code\git\tests\sample_data
Sample PDB path: c:\Users\hlern\code\git\tests\sample_data\sample.pdb
Expected CRD path: c:\Users\hlern\code\git\tests\sample_data\expected_output.crd


In [5]:
def test_conversion_output():
    # Verify test files exist first
    if not os.path.exists(SAMPLE_PDB):
        raise FileNotFoundError(f"Test PDB file not found at: {SAMPLE_PDB}")
    if not os.path.exists(EXPECTED_CRD):
        raise FileNotFoundError(f"Expected CRD file not found at: {EXPECTED_CRD}")
    
    print(f"Using PDB file: {SAMPLE_PDB}")
    print(f"Comparing against: {EXPECTED_CRD}")
    
    # Convert the sample PDB
    converted_content = convert_pdb_to_crd(SAMPLE_PDB)
    
    # Read expected output
    with open(EXPECTED_CRD, 'r') as f:
        expected_content = f.read()
    
    # Compare line by line
    converted_lines = converted_content.splitlines()
    expected_lines = expected_content.splitlines()
    
    assert len(converted_lines) == len(expected_lines), \
        f"Line count mismatch: {len(converted_lines)} vs {len(expected_lines)}"
    
    for i, (conv_line, exp_line) in enumerate(zip(converted_lines, expected_lines)):
        assert conv_line == exp_line, \
            f"Line {i+1} mismatch:\nConverted: {conv_line}\nExpected: {exp_line}"
    
    print("✅ Conversion output matches expected result")

# Run the test
test_conversion_output()

FileNotFoundError: Test PDB file not found at: c:\Users\hlern\code\git\tests\sample_data\sample.pdb

In [ ]:


"""
## Test 2: Verify File Saving Functionality
This test checks if the save_crd function correctly writes to a file.
"""

def test_file_saving():
    # Clean up any previous test output
    if os.path.exists(TEST_OUTPUT):
        os.remove(TEST_OUTPUT)
    
    # Save the CRD file
    output_path = save_crd(SAMPLE_PDB, TEST_OUTPUT)
    
    # Verify file was created
    assert os.path.exists(output_path), "Output file was not created"
    
    # Verify content matches
    with open(output_path, 'r') as f:
        saved_content = f.read()
    
    with open(EXPECTED_CRD, 'r') as f:
        expected_content = f.read()
    
    assert saved_content == expected_content, "Saved file content doesn't match expected"
    
    print(f"✅ File saved correctly to {output_path}")


test_file_saving()


In [ ]:

"""
## Test 3: Verify Error Handling
This test checks if the converter properly handles error cases.
"""


def test_error_handling():
    # Test non-existent file
    try:
        convert_pdb_to_crd("nonexistent.pdb")
        assert False, "Should have raised an error for non-existent file"
    except ValueError as e:
        assert "Could not read PDB file" in str(e)
    
    # Test empty PDB file
    empty_pdb = os.path.join(TEST_DIR, "empty.pdb")
    open(empty_pdb, 'w').close()  # Create empty file
    
    try:
        convert_pdb_to_crd(empty_pdb)
        assert False, "Should have raised an error for empty PDB"
    except ValueError as e:
        assert "No ATOM/HETATM records" in str(e)
    
    os.remove(empty_pdb)
    
    # Test malformed PDB file
    malformed_pdb = os.path.join(TEST_DIR, "malformed.pdb")
    with open(malformed_pdb, 'w') as f:
        f.write("ATOM      1  N   ALA A   1      28.490  31.600  33.379  1.00  0.00\n")
        f.write("ATOM      2  CA  ALA A   1      27.558  30.490  33.700  1.00  0.00\n")
        f.write("ATOM      3  C   ALA A   1      27.981  29.744  34.949  1.00  0.00\n")
        f.write("ATOM      4  O   ALA A   1      29.172  29.670  35.276  1.00  0.00\n")
        f.write("ATOM      BAD  O   ALA A   1      29.172  29.670  35.276  1.00  0.00\n")  # Malformed line
    
    try:
        convert_pdb_to_crd(malformed_pdb)
        assert False, "Should have raised an error for malformed PDB"
    except ValueError as e:
        assert "Error parsing line" in str(e)
    
    os.remove(malformed_pdb)
    
    print("✅ Error handling works as expected")

# Run the test
test_error_handling()

In [ ]:

"""
## Test 4: Verify Segment ID Functionality
This test checks if the segment ID parameter works correctly.
"""


def test_segid_functionality():
    # Convert with default segid
    default_content = convert_pdb_to_crd(SAMPLE_PDB)
    assert "HETA" in default_content, "Default segid not found in output"
    
    # Convert with custom segid
    custom_content = convert_pdb_to_crd(SAMPLE_PDB, seg_id="PROT")
    assert "PROT" in custom_content, "Custom segid not found in output"
    assert "HETA" not in custom_content, "Default segid found when custom was specified"
    
    print("✅ Segment ID functionality works as expected")

# Run the test
test_segid_functionality()


"""
## Test 5: Verify Automatic Output Naming
This test checks if the automatic output naming works correctly.
"""


def test_auto_output_naming():
    # Clean up any previous test output
    expected_auto_path = SAMPLE_PDB.replace(".pdb", ".crd")
    if os.path.exists(expected_auto_path):
        os.remove(expected_auto_path)
    
    # Save with automatic naming
    output_path = save_crd(SAMPLE_PDB)
    
    # Verify path is correct
    assert output_path == expected_auto_path, \
        f"Expected {expected_auto_path}, got {output_path}"
    
    # Verify file was created
    assert os.path.exists(output_path), "Output file was not created"
    
    # Clean up
    os.remove(output_path)
    
    print("✅ Automatic output naming works as expected")

# Run the test
test_auto_output_naming()


In [ ]:
"""
Cleanup
"""

if os.path.exists(TEST_OUTPUT):
    os.remove(TEST_OUTPUT)

print("✅ All tests completed successfully")